# Create my own dataset.

In [7]:
### LIBRARIES ###
import re
import time
import os
import openai
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import asyncio
import aiohttp
import pandas as pd
import requests
import json
import re
import pdfkit
import json
import sys
import time
import pandas as pd
from user_agent import generate_user_agent
from sentence_transformers import SentenceTransformer, CrossEncoder, util
from pdfminer.high_level import extract_text
from tqdm import tqdm
tqdm.pandas()

In [26]:
def content_embedding(content, model_name):
    '''
    generating embedding for model
    '''
    #Load the model
    # model = SentenceTransformer('sentence-transformers/' + model_name)
    model = SentenceTransformer(model_name,)

    embeddings = model.encode(content)
    return embeddings

mymodel = '/Users/slidarey/Downloads/rbc-demo/90000'

def split_into_sentences(content, no_sntcs=4):
    '''
    This function splits the text into sentences and then groups them into desired number of sentences.
    :param content: text to be split
    :param no_sntcs: number of sentences to be grouped
    :param split_overlap: if True, then the sentences will be split with overlap
    :param no_of_tokens: number of tokens to be grouped
    :param overlap: number of sentences to be overlapped
    :return: list of dictionaries with grouped sentences
    '''
    sentences = re.split('\.\s([A-Z][\w\d]+)', content)
    clnd_sntc = list()
    grpd_sntc = list()

    # splitting the whole text into sentences
    for i in range(len(sentences)):
        if i == 0:
            # first sentence
            clnd_sntc.append(sentences[i].strip() + '.')
        elif (i+2 == len(sentences)):

            # for last sentence = appending captured split pattern to it's sentence
            clnd_sntc.append((sentences[i] + sentences[i+1]).strip())
        elif ((i % 2) != 0):

            # for middle sentences - appending captured split pattern to it's sentence
            clnd_sntc.append(sentences[i] + sentences[i+1] + '.')
        else:
            pass

    # check if number of sentences is more than the desired number of sentences for grouping
    if len(clnd_sntc) >= no_sntcs:

        # to know how many group of sentences can be formed
        for i in range(len(clnd_sntc)//no_sntcs):

            # group and append desired set of sentences
            header = ' '.join(clnd_sntc[(i*no_sntcs):((i+1)*no_sntcs)])
            grpd_sntc.append(header)

            # check if remaining length is less than the desired number of group of sentences
            if (((len(clnd_sntc) - ((i+1)*no_sntcs)) < no_sntcs) and ((len(clnd_sntc) - ((i+1) * no_sntcs)) != 0)):

                # append remaining sentences
                header = ' '.join(clnd_sntc[((i+1)*no_sntcs):])
                grpd_sntc.append(header)
            else:
                pass
    else:
        # return all sentence as 1 group because total number of sentence is not up to desired number of group of sentences
        header = ' '.join(clnd_sntc)
        grpd_sntc.append(header)

    return grpd_sntc

def create_df(list_of_texts):
    '''
    Function to create dataframe from the series of list of dictionaries.
    '''
    master_list = list(list_of_texts)

    df = pd.DataFrame(master_list, columns=['texts'])
    return df

In [9]:
def get_filing(filing_ticker, filing_year):
    # debugging
    print('Getting your File')

    # Generate a user agent string for Chrome
    user_agent = generate_user_agent(navigator='chrome')

    lookup_headers = {
        "referer": "https://sec-api.io",
        "user-agent": user_agent
    }

    filing_ticker = str(filing_ticker)
    filing_year = int(filing_year)
    file_link = str()
    company_name = str()
    company_ticker = str()
    filing_id = str()
    data = {
            "query": {"query_string": {
                                        "query": "formType:\"10-K\" AND ticker:(" + filing_ticker + ") AND filedAt:[" + str(
                                        filing_year) + "-01-01 TO " + str(filing_year+1) + "-01-01] AND documentFormatFiles.type:*21*"
                                        }
                    }, "sort": [{
                                    "filedAt": {"order": "desc"}
                                    }]}

    try:
        response = requests.post(
            "https://api.sec-api.io/", headers=lookup_headers, json=data, timeout=10)
    except requests.exceptions.Timeout:
        print('The request timed out')

    if response.status_code == 200:
        parsed_json = json.loads(response.text)
        hits = parsed_json['total']['value']
        if hits > 0:
            company_name = parsed_json['filings'][0]['companyName']
            company_ticker = parsed_json['filings'][0]['ticker']
            examine_link = parsed_json['filings'][0]['linkToFilingDetails']
            filing_id = parsed_json['filings'][0]['id']
            if examine_link.find('ix?doc=/') != -1:
                file_link = examine_link.replace('ix?doc=/', '')
            else:
                file_link = examine_link
        else:
            print('No hits')
            return """ <p style="text-align: left; font-size: 12px; color: gray; font-style: italic;"> The file you're looking for doesn't exist yet. </p> """
    else:
        print(response.status_code)

    # getting the html content of the 10-K filing and removing any image content and table content

    html_to_pdf_retries = 0
    get_page_headers = {"user-agent": user_agent}

    try:
        response = requests.get(file_link, headers=get_page_headers, timeout=10)
    except requests.exceptions.Timeout:
        print('The request timed out')

    if response.status_code == 200:
        html = response.text
        html_without_images = re.sub(r'<img.*?>', '', html)
        html_without_tables = re.sub(r'<table.*?>.*?</table>', '', html_without_images)
        print(response.status_code)
    else:
        print(response.status_code)

    #####################################################
    # changed html_without_tables to html_without_images
    pdfkit.from_string(html_without_images, 'output.pdf', options={"enable-local-file-access": ""})

    PDF_read = extract_text('output.pdf')
    e_PDF_read = PDF_read.replace('\n\n', ' ').replace('\n', ' ')
    e_PDF_read = re.sub(r'\s+', ' ', e_PDF_read)

    # grouped_sentences = split_into_sentences(concatenated_output)
    grouped_sentences = split_into_sentences(e_PDF_read)
    df_texts = create_df(grouped_sentences)

    def create_context(content):
        '''
        Function to create context from the dataset.
        '''
        context = 'Company: ' + company_name + \
            ' (' + company_ticker + ')' + '\n' + content['texts']
        return context

    # creating context using the company_name and new_text column
    df_texts['context'] = df_texts.apply(create_context, axis=1)

    # creating context using the company_name and new_text column
    # df_texts['embedding'] = df_texts['context'].progress_apply(content_embedding, model_name=mymodel)
    df_texts['filing_id'] = filing_id
    df_texts['company_name'] = company_name
    df_texts['ticker'] = company_ticker
    df_texts['year_of_filing'] = filing_year
    df_texts.drop(columns=['texts'], inplace=True)

    # merge the new dataframe with the existing dataframe
    init_data = pd.read_parquet('data.parquet')
    merge_df = pd.concat([init_data, df_texts], ignore_index=True)
    merge_df.to_parquet('data.parquet', compression='gzip')

    return """ <p style="text-align: left; font-size: 12px; color: gray; font-style: italic;"> Filing parsed. </p> """

In [10]:
df = pd.DataFrame(columns=['filing_id', 'ticker', 'year_of_filing', 'company_name', 'context'])
df.to_parquet('data.parquet', compression='gzip')

In [ ]:
# apply the get filing function using different tickers and years
tickers = ['AAPL', 'MSFT', 'AMZN', 'NFLX']
years = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
for ticker in tickers:
    for year in years:
        get_filing(ticker, year)
        print(ticker, year)


In [24]:
t = pd.read_parquet('data.parquet')
t.head()

,filing_id,ticker,year_of_filing,company_name,context
0,b5c5243ad175f6c2580d720e74e2aed1,GOOG,2016,Alphabet Inc.,Company: Alphabet Inc. (GOOG)\n10-K 1 goog10-k...
1,b5c5243ad175f6c2580d720e74e2aed1,GOOG,2016,Alphabet Inc.,Company: Alphabet Inc. (GOOG)\nYes ý Yes ¨ No ...
2,b5c5243ad175f6c2580d720e74e2aed1,GOOG,2016,Alphabet Inc.,Company: Alphabet Inc. (GOOG)\nGoogle Inc. Yes...
3,b5c5243ad175f6c2580d720e74e2aed1,GOOG,2016,Alphabet Inc.,Company: Alphabet Inc. (GOOG)\nYes ý Yes ý No ...
4,b5c5243ad175f6c2580d720e74e2aed1,GOOG,2016,Alphabet Inc.,Company: Alphabet Inc. (GOOG)\nGoogle Inc. Lar...


In [25]:
t.shape

(15064, 5)

In [27]:
t['embedding'] = t['context'].progress_apply(content_embedding, model_name=mymodel)

100%|██████████| 15064/15064 [6:46:55<00:00,  1.62s/it]  


In [28]:
t.head()

,filing_id,ticker,year_of_filing,company_name,context,embedding
0,b5c5243ad175f6c2580d720e74e2aed1,GOOG,2016,Alphabet Inc.,Company: Alphabet Inc. (GOOG)\n10-K 1 goog10-k...,"[-0.062086843, -0.19040565, -0.34217635, 0.019..."
1,b5c5243ad175f6c2580d720e74e2aed1,GOOG,2016,Alphabet Inc.,Company: Alphabet Inc. (GOOG)\nYes ý Yes ¨ No ...,"[-0.06756891, -0.11042685, -0.30150792, -0.147..."
2,b5c5243ad175f6c2580d720e74e2aed1,GOOG,2016,Alphabet Inc.,Company: Alphabet Inc. (GOOG)\nGoogle Inc. Yes...,"[0.164681, -0.3119648, -0.32115456, -0.1062936..."
3,b5c5243ad175f6c2580d720e74e2aed1,GOOG,2016,Alphabet Inc.,Company: Alphabet Inc. (GOOG)\nYes ý Yes ý No ...,"[-0.019733991, -0.41353232, -0.31599152, -0.10..."
4,b5c5243ad175f6c2580d720e74e2aed1,GOOG,2016,Alphabet Inc.,Company: Alphabet Inc. (GOOG)\nGoogle Inc. Lar...,"[-0.14395475, -0.5422194, -0.30889645, 0.01593..."


In [30]:
t.to_parquet('data copy.parquet', compression='gzip')